# Primitives examples

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: -0.22127659574468084
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.21030927835051547
>>> Standard errors for PUB 0: 0.11287983317881108
>>> Expectation values for PUB 1: -0.20981842636180228
>>> Standard errors for PUB 1: 0.18300226543012751
>>> Expectation values for PUB 2: -0.1362776025236593
>>> Standard errors for PUB 2: 0.1556157713292338


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 0.99673974  0.94632751  0.79613672  0.57064135  0.29473375 -0.02551988
  -0.31251399 -0.59072257 -0.82333004 -0.95594976 -0.99025518 -0.9454908
  -0.79048888 -0.56666694 -0.28615739  0.02217301  0.31209563  0.61394148
   0.82102906  0.95030192  0.99569385]
 [ 0.01275994  0.32171789  0.60682938  0.82688609  0.96117925  1.00259677
   0.93984295  0.79550918  0.56520268  0.28720329  0.00648456 -0.31816184
  -0.6055743  -0.82249332 -0.95825074 -1.00573446 -0.93879706 -0.80471307
  -0.56645776 -0.3164884   0.02280055]
 [-0.02426481 -0.30582025 -0.60097236 -0.81621794 -0.95281207 -0.99945908
  -0.95364879 -0.80094784 -0.58047278 -0.29577964  0.00669374  0.32088117
   0.58779406  0.81203435  0.95197536  0.99548467  0.95260289  0.79864687
   0.58256457  0.32004445  0.00753046]
 [ 1.0053161   0.95427633  0.80617733  0.59030421  0.29410621 -0.01861696
  -0.30644779 -0.6137323  -0.81998317 -0.95260289 -1.00050497 -0.94067967
  -0.78379513 -0.57168724 -0.29410621  0.02238

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/guides/execution-modes#job-mode) or [batch mode](/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [4]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.16927322907083717
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.04192472606002858
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['0010111010000111101000110011010010010100111000000000000011001110011101000001111110001110011000110110000100000011110001001100101', '1000111011000001110101001011111000000111100001000011001011010111011110100010010010011000011101100110101100001011000100100100101', '0011111110001110000011010101110100000110011101000100100011100001000010111100101001100011000000000110000110011100101001100110001', '1001011110000101110110000110110001100000000011100000111011110110110001100110100110000100110001001101000000000010100000111100010', '0101101011111101110011111110010000110110011001101011001001100110100010011001011010100111100001100110111110001000111101010110001', '1001001110000000110101000001110001000100111100111110101100001110001000000000100111101011000010110110000100110110001001110100000', '1101001001111011110110110111100110001000011001010001101111010111100100110111101010010101111000100110010011011111111101000100001', '10110100110011001100100111110001010011011110110110010001000

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [IQP(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0110100001010011111001000010011110011111010001100100011100011000111000111011110010011001010000001011001101110101100110000000100', '1010101111101100101010111000111111000110000010100101001000110000011010110011000000010000001001001000101000100100100100001010001', '0110011011001000010110101010111111111001111100101101100000100000111100100110111010011010001000000000101011101100101100000000100', '1011100010001101011000001111111100001001000001100011000100000010010000100000011010011010100001000010001011010100110100000101000', '0001001001000101011100001110110000000100000000000011110010010111011110110011100001010000001010000010000001000000111000000100001', '0000101000010001100101110110001101110010000100101110000011011010111000111110101110010010000011000000010111011101001100101110000', '1011011010001010000011101110111110001000000101100111001011001110011111000100101000001101101100001111101010110000110000101010000', '1001111010100100100100101001111010010000100000101

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import RealAmplitudes
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = RealAmplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0111001110000111100111010110110010011111000000001011000000101110000000110111010011000101001110100011010010000000011010110011111', '1011000101110011111011000110100111110011000001010101000111111011111000101100000100001110100100010000010100001101001010110000110', '0010000000110101011111000101000010100010100000011100000000010011101011100110101001000100001110100010011101000000000110110001101', '0101011100000101011110100101010010100001000000100111001011111110110110100101000110001110111101100100110001111100001111101000101', '0010111000000011000000101000100001110101000001000000100111111100011111100100000110010000001101100101011101111110010011011101101', '0111110011011100001111100010110110101110001010001010100110001111000000101110000100011101000100100010010100101010011000111100011', '0000101000111011001011000010110101100101100000010011001000101000101011001111011100101001010001010100001100111100010011111000101', '01011100100101111110010001101

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/guides/execution-modes#job-mode) or [batch mode](/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [8]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1000101011111010100001000110111101000100010000010010101101010100110000000111001100000001111101000010101000010110111010001101000', '0100010100011010110010000011010110001010001101011111101000000110111010110111110101111101100010000010011100010100000000100111001', '1000100010010101011011011010111100001000100000100010101011011100011100110010111011101000010110000110100001010110100100000101001', '1001001000101111011111000010001100000001111000001111010101000111001011100000111011110000000100101011111000000001000011001100000', '1001010110000101010000000010100100010000000000100111000010010000011010100011011001110010000000101111000011000110000001000101000', '0001111000011110001110101000101110001100000111000110001010010000001100000101001110000100001011100011001001001011110000100101000', '0100111101100000101001100010110010101010110010001010101000110001010000100111000101001101001100100010000001001000100010010010000', '0100010101010111110101111100110000

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1011010000101011010010010110011110000100101100000000001110000110010100001100001100010010011000111100100011000101000101000100001', '0011010011100110111001011010011110110000110001010001000100011001110110000100111100110010010110010101000010110001010100011000010', '1100011100000110001100100000100000001000000110000001001111011010111100100000011010010010000101100000001111111001010111010100000', '0000011011010100000110110010010110101010101010000101100000100111000101010100000101001100111000010001011101110011011101100100001', '0101000111010111101000010000100100011100111010000001001100010011101100100001010111100000000110011011000000000100010111010100011', '0111101100111101000001001001010001101100100110010100010110011000011110100100000101110000000110010100011100000000010011010110001', '1101110010000011101111010000010111000001001101100001101010001011010000100001000101011010000000100000001000110100000000010100010', '1110100110010101011110000110000100

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>